In [1]:
import numpy as np
import pandas as pd
import graphviz
import altair as alt

In [77]:
import sklearn
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn import tree 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, SVR
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.preprocessing import normalize, scale, Normalizer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier #installation guide: https://pypi.org/project/lightgbm/

In [36]:
def print_scores(model, 
                X_train, y_train,
                X_test, y_test):
    """
    Prints the train and test accuracies 
    Parameters
    ----------
    model: sklearn classifier
        
    X_train: numpy.ndarray        
        The X train set
    y_train: numpy.ndarray
        The y train set    
    X_valid: numpy.ndarray        
        The X test set
    y_valid: numpy.ndarray
        The y test set    
    Returns
    -------
        prints the train and test accuracies. 
            
    """    
    print("Training error:   %.2f" % (model.score(X_train, y_train)))
    print("Validation error: %.2f" % (model.score(X_test, y_test)))

In [3]:
heart = pd.read_csv("heart.csv")
heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


## Explanation of Variables

1. age - age in years 
2. sex - (1 = male; 0 = female) 
3. cp - chest pain type 
    - cp: chest pain type 
    - Value 1: typical angina 
    - Value 2: atypical angina 
    - Value 3: non-anginal pain 
    - Value 4: asymptomatic 
4. trestbps - resting blood pressure (in mm Hg on admission to the hospital) 
5. chol - serum cholestoral in mg/dl 
6. fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) 
7. restecg - resting electrocardiographic results 
    - Value 0: normal 
    - Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV) 
    - Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria 
8. thalach - maximum heart rate achieved 
9. exang - exercise induced angina (1 = yes; 0 = no) 
10. oldpeak - ST depression induced by exercise relative to rest 
11. slope - the slope of the peak exercise ST segment 
    - Value 1: upsloping 
    - Value 2: flat 
    - Value 3: downsloping 
12. ca - number of major vessels (0-3) colored by flourosopy 
13. thal 
    - 3 = normal
    - 6 = fixed defect
    - 7 = reversable defect 
14. target - have disease or not (1=yes, 0=no)

There are 4 dummy variables: sex, fasting blood sugar, exercise induced angina, and the 'target' variable.

In [4]:
heart.describe(include = 'all')

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [5]:
heart.isnull()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,False,False,False,False,False,False,False,False,False,False,False,False,False,False
299,False,False,False,False,False,False,False,False,False,False,False,False,False,False
300,False,False,False,False,False,False,False,False,False,False,False,False,False,False
301,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [6]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
age         303 non-null int64
sex         303 non-null int64
cp          303 non-null int64
trestbps    303 non-null int64
chol        303 non-null int64
fbs         303 non-null int64
restecg     303 non-null int64
thalach     303 non-null int64
exang       303 non-null int64
oldpeak     303 non-null float64
slope       303 non-null int64
ca          303 non-null int64
thal        303 non-null int64
target      303 non-null int64
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [7]:
#Determine if there ANY entry with missing value
heart.isnull().values.any()

False

In [8]:
heart.groupby('target').mean()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
target,,,,,,,,,,,,,
0,56.601449,0.826087,0.478261,134.398551,251.086957,0.159420,0.449275,139.101449,0.550725,1.585507,1.166667,1.166667,2.543478
1,52.496970,0.563636,1.375758,129.303030,242.230303,0.139394,0.593939,158.466667,0.139394,0.583030,1.593939,0.363636,2.121212


In [9]:
heart.groupby('sex').mean()

,age,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
sex,,,,,,,,,,,,,
0,55.677083,1.041667,133.083333,261.302083,0.12500,0.572917,151.125000,0.229167,0.876042,1.427083,0.552083,2.125000,0.750000
1,53.758454,0.932367,130.946860,239.289855,0.15942,0.507246,148.961353,0.371981,1.115459,1.386473,0.811594,2.400966,0.449275


In [10]:
X = heart.drop(columns = ['target'])
X.head(2)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2


In [11]:
y = heart['target']
y

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 100)

In [67]:
heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [68]:
heart['cp'].unique()

array([3, 2, 1, 0])

In [69]:
# Identify the categorical and numeric columns
numeric_features = ['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalach', 'exang', 'oldpeak', 'ca']
categorical_features = ['cp', 'slope', 'thal', 'restecg']

In [70]:
# Create the transformer for numeric and categorical variables
preprocessor = ColumnTransformer(transformers=[
    ('minmax', MinMaxScaler(), numeric_features),
    ('ohe', OneHotEncoder(drop='first'), categorical_features)])

In [71]:
# Apply data transformations and convert back to dataframe
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),
                       index=X_train.index,
                       columns=(numeric_features +
                                list(preprocessor.named_transformers_['ohe']
                                     .get_feature_names(categorical_features))))
X_test = pd.DataFrame(preprocessor.transform(X_test),
                      index=X_test.index,
                      columns=X_train.columns)

In [72]:
(numeric_features +
                                list(preprocessor.named_transformers_['ohe']
                                     .get_feature_names(categorical_features)))

['age',
 'sex',
 'trestbps',
 'chol',
 'fbs',
 'thalach',
 'exang',
 'oldpeak',
 'ca',
 'cp_1',
 'cp_2',
 'cp_3',
 'slope_1',
 'slope_2',
 'thal_1',
 'thal_2',
 'thal_3',
 'restecg_1',
 'restecg_2']

In [73]:
X_train.head()

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,ca,cp_1,cp_2,cp_3,slope_1,slope_2,thal_1,thal_2,thal_3,restecg_1,restecg_2
184,0.437500,1.0,0.571429,0.402062,0.0,0.435115,0.0,0.419355,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
19,0.833333,0.0,0.469388,0.388316,0.0,0.610687,0.0,0.290323,0.50,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
118,0.354167,0.0,0.112245,0.268041,0.0,0.770992,0.0,0.000000,0.00,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
41,0.395833,1.0,0.367347,0.408935,0.0,0.832061,0.0,0.032258,0.00,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
59,0.583333,0.0,0.346939,0.608247,0.0,0.671756,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [74]:
# Create a simple voting classifier using scikit-learn's VotingClassifier

classifiers = {
    "Decision tree"         : DecisionTreeClassifier(max_depth=5),
    "KNN"                   : KNeighborsClassifier(),
    "Naive Bayes"           : GaussianNB(),
    "Logistic Regression"   : LogisticRegression(),
    "Random Forest"         : RandomForestClassifier(),
    "SVM"                   : SVC(probability=True)
}


ensemble = VotingClassifier(classifiers.items(), voting="soft")

ensemble.fit(X_train, y_train);
print(ensemble.score(X_train, y_train))
print(ensemble.score(X_test, y_test))

0.9049586776859504
0.8852459016393442


In [84]:
print('Ensemble performance: \n')
print('Train_error: %0.3f; Test error: %0.3f'%(train_accuracy, test_accuracy))

Ensemble performance: 

Train_error: 0.880; Test error: 0.852


In [76]:
for model_name, estimator in ensemble.named_estimators_.items():
    train_accuracy = (estimator.score(X_train, y_train))
    test_accuracy = (estimator.score(X_test, y_test))
    print('%s: Train_error: %0.3f; Test error: %0.3f'%(model_name, train_accuracy, test_accuracy))

Decision tree: Train_error: 0.926; Test error: 0.820
KNN: Train_error: 0.855; Test error: 0.869
Naive Bayes: Train_error: 0.839; Test error: 0.902
Logistic Regression: Train_error: 0.851; Test error: 0.885
Random Forest: Train_error: 1.000; Test error: 0.836
SVM: Train_error: 0.880; Test error: 0.852


In [79]:
accuracies = []
for model_name, estimator in ensemble.named_estimators_.items():
    train_accuracy = (estimator.score(X_train, y_train))
    test_accuracy = (estimator.score(X_test, y_test))
    print('%s: Train_error: %0.3f; Test error: %0.3f'%(model_name, train_accuracy, test_accuracy))
    
    accuracies.append([model_name, estimator.score(X_train, y_train), estimator.score(X_test, y_test)])

Decision tree: Train_error: 0.926; Test error: 0.820
KNN: Train_error: 0.855; Test error: 0.869
Naive Bayes: Train_error: 0.839; Test error: 0.902
Logistic Regression: Train_error: 0.851; Test error: 0.885
Random Forest: Train_error: 1.000; Test error: 0.836
SVM: Train_error: 0.880; Test error: 0.852


In [80]:
accuracies_df = pd.DataFrame(data = accuracies, columns = ['model', 'train_accuracies', 'test_accuracies'])
accuracies_df

,model,train_accuracies,test_accuracies
0,Decision tree,0.925620,0.819672
1,KNN,0.855372,0.868852
2,Naive Bayes,0.838843,0.901639
3,Logistic Regression,0.851240,0.885246
4,Random Forest,1.000000,0.836066
5,SVM,0.880165,0.852459


In [ ]:
# Define models
tree_models = {
    'DT': GridSearchCV(DecisionTreeClassifier(),
                       param_grid = {'max_depth': [1, 3, 6, 9, None]},
                       cv = 5),
    'RF': GridSearchCV(RandomForestClassifier(n_estimators = 100),
                       param_grid = {'max_depth': [1, 3, 6, 9, None]},
                       cv = 5),
    'GB': GridSearchCV(GradientBoostingClassifier(n_estimators = 100),
                       param_grid = {'learning_rate': [0.01, 0.1, 0.5],
                                     'max_depth': [1, 3, 6, 9, None]},
                       cv = 5)}
# Prediction loop
for name, model in tree_models.items():
    tree = make_pipeline(StandardScaler(),
                         model)
    tree.fit(X_train, y_train)
    print(f"{name} best hyperparams = {model.best_params_}")
    print(f"{name} has train error = {1 - tree.score(X_train, y_train):.2f}")
    print(f"{name} has test error = {1 - tree.score(X_test, y_test):.2f}")